## Working with git

In [ ]:
#| default_exp git

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import os
from pydantic_settings import BaseSettings
from pathlib import Path

# Load environment variables from .env file
from dotenv import load_dotenv; load_dotenv();

In [ ]:
#| export
from lovely_docs.settings import GitSource, settings
from git import Repo, InvalidGitRepositoryError, NoSuchPathError

In [ ]:
#| export

def git_progress(op_code, cur_count, max_count=None, message=''):
    if max_count:
        print(f"\r{op_code}: {cur_count}/{max_count} {message}", end='', flush=True)
    else:
        print(f"\r{op_code}: {cur_count} {message}", end='', flush=True)


def clone_repo(source: GitSource):
    """Clone a git repository if it doesn't exist, or ensure an existing one is clean and at the specified commit.

    Args:
        source (GitSource): Configuration object containing the repository URL, name, and target commit.

    Returns:
        None

    Will raise exceptions on errors
    """
    clone_dir = settings.git_dir / source.name

    try:
        repo = Repo(clone_dir)
        repo.git.clean('-fdq') # Just in case
    except (InvalidGitRepositoryError, NoSuchPathError):
        repo = Repo.clone_from(source.repo, clone_dir, progress=git_progress)

    repo.git.reset('--hard', source.commit)
    return repo.head.commit.hexsha